# Dynamics: simulation and control
This notebook focuses on the simulation of polyarticulated systems, with final exercices where simple control laws have to be designed. The first part deals with collision detection using the hppfcl module of pinocchio. We then build a complete simulation engine for rigid unilateral contacts. 

In [1]:
import gepetuto.magic

NB: as for all the tutorials, a magic command %do_not_load is introduced to hide
    the solutions to some questions. Change it for %load if you want to see (and
    execute) the solution.


## Set up

We will use several models in this tutorial:
- a simple scene with 3 convex objects (buildSceneThreeBodies)
- a variation of this first scene with more objects and walls (buildScenePillsBox)
- a stack of cubes of various size (buildSceneCubes)
- a robot hand

In [2]:
from tp4.scenes import buildSceneThreeBodies, buildScenePillsBox, buildSceneCubes

We rely on the HPP-FCL module of Pinocchio, which compute collision between geometries.

In [3]:
import hppfcl
import pinocchio as pin
import numpy as np
import time
from supaero2024.meshcat_viewer_wrapper import MeshcatVisualizer

## Starting with collisions

The goad of this section is to introduce a simple example of collision distances between bodies, and the underlying notions of witness points and segment and normal direction.

Let's build a simple scene with 3 objects and display their proximity.

In [4]:
# %load tp4/generated/example_display_witness_build
# Build a scene
model,geom_model = buildSceneThreeBodies()
data = model.createData()
geom_data = geom_model.createData()

# Start meshcat
viz = MeshcatVisualizer(model=model, collision_model=geom_model,
                        visual_model=geom_model,url="classical")


*** You asked to start meshcat "classically" in tcp://127.0.0.1:6000
*** Did you start meshcat manually (meshcat-server)
Wrapper tries to connect to server <tcp://127.0.0.1:6000>
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/


In [5]:
q=pin.randomConfiguration(model)
viz.display(q)

In [6]:
viz.viewer.jupyter_cell()

We can compute the distances between the 3 objects. Let's do it and show the closest points.

In [7]:
from tp4.display_witness import DisplayCollisionWitnessesInMeshcat

In [8]:
# %load tp4/generated/example_display_witness_witness
# Build the viewer add-on to display the witnesses.
mcWitnesses = DisplayCollisionWitnessesInMeshcat(viz)


In [9]:
pin.computeDistances(model,data,geom_model,geom_data,q)
mcWitnesses.displayDistances(geom_data)

Each collision pair corresponds to a pair of closets points, respectively located at the surface of the collision geometries if they are not colliding. These points are sometime called the *witness* points.

The witness points are connected by the *witness* segment. This segment is normal to the two collision surfaces in the case the surface is smooth around the witness point. The normalized direction is called the collision *normal*. Its orientation is a convention (the most logical convention is to go from body 1 to body 2 of the collision pair).

Let's move the objects to better see these witness elements.

In [10]:
# %load tp4/generated/example_display_witness_trajectory
v = (np.random.rand(model.nv)*2-1)*1e-3
r0 = [ np.linalg.norm(q[7*i:7*i+3]) for i in range(model.nq//7) ]
for t in range(100):

    # Update the robot position along an arbitrary trajectory
    q = pin.integrate(model,q,v*10)
    for i in range(model.nq//7):
        q[7*i:7*i+3] *= r0[i]/np.linalg.norm(q[7*i:7*i+3])
    viz.display(q)

    # Display the witness points
    pin.computeDistances(model,data,geom_model,geom_data,q)
    mcWitnesses.displayDistances(geom_data)

    time.sleep(.01)


## Details about Pinocchio HPP-FCL 